In the [previous]() blog post, we saw how Linkin Park's musical style changed over the years, by looking at the mean and variation of audio features of their songs. In paricular, we looked at 4 audio features - duration, energy, loudness and valence. In this post, we'll try to relate some of these features to the general "mood" of LP's songs, and how these moods changed through the years. You don't need to read the previous post to understand this one - I am only using 2 of the 4 previous audio features - energy and valence. Also, I will be explaining these features again. 

Before we start the analysis, though, let me try to articulate what I mean when I say "mood" of a song.

## Mood and Music

If you're anything more than a casual listener of music, you will find a strong correlation between a song and the emotions it invokes in you. Some hardcore listeners might even experience using music as a tool to manipulate or "hack" their emotions - listening to a particular song to switch from one mood to the next. For example - some people might play highly energetic music in their room when they're feeling lethargic, to get some momentum and start doing things. After a long day at work, another person might put on some relaxing "peaceful" music to switch from a hyper focused state to a more relaxed state of mind. "Somber" or "neutral" music is usually considered to be suited to listening while working, since it takes you into an objective state of mind. \
The [Gracenote Music API] 

## Dataset

We'll be using the [Gracenote Music Web API](https://developer.gracenote.com/web-api) to get mood information for Linkin Park's songs. [This blog post](http://www.gracenote.com/computing-musical-mood-at-gracenote/) contains more detailed information on the algorithms behind classification of mood of each song. We will be using a subset of the 25 moods used by Gracenote. Following are the moods we'll be using:

* Peaceful
* Sentimental
* Upbeat
* Sensual
* Serious
* Somber
* Exciting
* Energizing
* Aggressive

For those that didn't read the previous post, here is a quick recap of the 2 audio features (energy and valence) we'll be using to relate to the "mood" of a song:

* Energy - a number between 0 and 1 which is a perceptual measure of intensity and activity of a song. According to Spotify's API documentation :
> Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale.

* Valence - A measure from 0.0 to 1.0 describing the musical positiveness conveyed by the track.
> Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).